# Lazy API

Polars supports two modes of operation: lazy and eager. The examples so far have used the eager API, in which the query is executed immediately. In the lazy API, the query is only evaluated once it is collected. Deferring the execution to the last minute can have significant performance advantages and is why the lazy API is preferred in most cases. Let us demonstrate this with an example:

In [23]:
import polars as pl

df = pl.read_csv("./Iris.csv")

df_small = df.filter(pl.col("sepal_length") > 5)
df_agg = df_small.group_by("species").agg(pl.col("sepal_width").mean())
print(df_agg)

shape: (3, 2)
┌─────────────────┬─────────────┐
│ species         ┆ sepal_width │
│ ---             ┆ ---         │
│ str             ┆ f64         │
╞═════════════════╪═════════════╡
│ Iris-setosa     ┆ 3.713636    │
│ Iris-virginica  ┆ 2.983673    │
│ Iris-versicolor ┆ 2.804255    │
└─────────────────┴─────────────┘


In this example we use the eager API to:

- Read the iris dataset.
- Filter the dataset based on sepal length.
- Calculate the mean of the sepal width per species.

Every step is executed immediately returning the intermediate results. This can be very wasteful as we might do work or load extra data that is not being used. If we instead used the lazy API and waited on execution until all the steps are defined then the query planner could perform various optimizations. In this case:

- Predicate pushdown: Apply filters as early as possible while reading the dataset, thus only reading rows with sepal length greater than 5.
- Projection pushdown: Select only the columns that are needed while reading the dataset, thus removing the need to load additional columns (e.g., petal length and petal width).

In [ ]:
q = (
    pl.scan_csv("../../datasets/Iris.csv")
    .filter(pl.col("sepal_length") > 5)
    .group_by("species")
    .agg(pl.col("sepal_width").mean())
)

df = q.collect()

These will significantly lower the load on memory & CPU thus allowing you to fit bigger datasets in memory and process them faster. Once the query is defined you call collect to inform Polars that you want to execute it. You can [learn more about the lazy API in its dedicated chapter](https://docs.pola.rs/user-guide/lazy/).

**Note**: In many cases the eager API is actually calling the lazy API under the hood and immediately collecting the result. This has the benefit that within the query itself optimization(s) made by the query planner can still take place.

## When to use which
In general, the lazy API should be preferred unless you are either interested in the intermediate results or are doing exploratory work and don't know yet what your query is going to look like.

## Previewing the query plan
When using the lazy API you can use the function `explain` to ask Polars to create a description of the query plan that will be executed once you collect the results. This can be useful if you want to see what types of optimizations Polars performs on your queries. We can ask Polars to explain the query `q` we defined above:

In [26]:
print(q.explain())

AGGREGATE[maintain_order: false]
  [col("sepal_width").mean()] BY [col("species")]
  FROM
  Csv SCAN [./Iris.csv]
  PROJECT 3/6 COLUMNS
  SELECTION: [(col("sepal_length")) > (5.0)]
  ESTIMATED ROWS: 170


Immediately, we can see in the explanation that Polars did apply predicate pushdown, as it is only reading rows where the sepal length is greater than 5, and it did apply projection pushdown, as it is only reading the columns that are needed by the query.

The function explain can also be used to see how expression expansion will unfold in the context of a given schema. Consider the example expression from the [section on expression expansion](https://docs.pola.rs/user-guide/concepts/expressions-and-contexts/#expression-expansion):

In [27]:
(pl.col(pl.Float64) * 1.1).name.suffix("*1.1")

<Expr ['[(cs.by_dtype([Float64])) * (d…'] at 0x71001BEB5520>

In [28]:
schema = pl.Schema(
    {
        "int_1": pl.Int16,
        "int_2": pl.Int32,
        "float_1": pl.Float64,
        "float_2": pl.Float64,
        "float_3": pl.Float64,
    }
)

print(
    pl.LazyFrame(schema=schema)
    .select((pl.col(pl.Float64) * 1.1).name.suffix("*1.1"))
    .explain()
)

SELECT [[(col("float_1")) * (1.1)].alias("float_1*1.1"), [(col("float_2")) * (1.1)].alias("float_2*1.1"), [(col("float_3")) * (1.1)].alias("float_3*1.1")]
  DF ["int_1", "int_2", "float_1", "float_2", ...]; PROJECT["float_1", "float_2", "float_3"] 3/5 COLUMNS
